In [1]:
import catboost
print(catboost.__version__)
import re
#import bamboolib as bam
import pandas as pd
import numpy as np
import sys
from sklearn import preprocessing
# ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.metrics import precision_score, recall_score, confusion_matrix,classification_report
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from datetime import date
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE,SMOTENC,SVMSMOTE
from imblearn.pipeline import make_pipeline
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support as score
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

1.0.3


In [2]:
#df = pd.read_csv(bam.titanic_csv)
df = pd.read_csv("D:/Projects/cp_heartdetect_model/Dataset/train.csv")
#df

In [ ]:
# print(df.head())
# df.info()

### Обработка колонок

In [3]:
df_test = df.rename(columns=
{'ID': 'id', 
'Пол': 'sex',
'Семья': 'family_status',
'Этнос': 'ethnos',
'Национальность': 'nationality',
'Религия': 'religion',
'Образование': 'education',
'Профессия': 'profession',
'Вы работаете?': 'work_status',
'Выход на пенсию': 'pension_age',
'Прекращение работы по болезни': 'stop_working',
'Сахарный диабет': 'diabetes',
'Гепатит': 'hepatitis',
'Онкология': 'oncology',
'Хроническое заболевание легких': 'lung_disease',
'Бронжиальная астма': 'bronchial_asthma',
'Туберкулез легких ': 'tuberculosis',
'ВИЧ/СПИД': 'HIV_AIDS',
'Регулярный прим лекарственных средств': 'intake_medicines',
'Травмы за год': 'injuries_per_year',
'Переломы': 'fractures',
'Статус Курения': 'smoke_status',
'Возраст курения': 'smoking_age',
'Сигарет в день': 'cigarettes_pd',
'Пассивное курение': 'passive_smoke',
'Частота пасс кур': 'pass_smoke_rate',
'Алкоголь': 'alcohol_status',
'Возраст алког': 'alcohol_age',
'Время засыпания': 'sleep_start_time',
'Время пробуждения': 'sleep_end_time',
'Сон после обеда': 'sleep_daily',
'Спорт, клубы': 'sport_status',
'Религия,клубы': 'religion_status',
'ID_y': 'id_y',
'Артериальная гипертензия': 'arterial_hypertension',
'ОНМК': 'insult_status',
'Стенокардия, ИБС, инфаркт миокарда': 'infarction_status',
'Сердечная недостаточность': 'heart_failure',
'Прочие заболевания сердца': 'oheart_diseases',
})

In [ ]:
df_test.columns

In [ ]:
# df_test.family_status.unique()
# df_test.ethnos.unique()
# df_test.nationality.unique()
# df_test.religion.unique()
# df_test.education.unique()
# df_test.profession.unique()
# df_test.work_status.unique()
for col in df_test:
    print(df_test[col].unique())

# Приведение датасета к нормальному виду

In [4]:
# replace empty with 0
#df_test = df_test.replace('', np.nan, regex=True)
df_test = df_test.fillna(0)

# sex 1-женщина, 0-мужчина
df_test.replace({'sex': 'М'}, {'sex': '0'}, regex=True, inplace = True)
df_test.replace({'sex': 'Ж'}, {'sex': '1'}, regex=True, inplace = True)

# family_status: 1-в браке в настоящее время, 0-в разводе, 2-вдовец / вдова, 3-никогда не был(а) в браке, 4-гражданский брак / проживание с партнером, 5-раздельное проживание (официально не разведены)
df_test.replace({'family_status': 'в браке в настоящее время'}, {'family_status': '1'}, regex=True, inplace = True)
df_test.replace({'family_status': 'в разводе'}, {'family_status': '0'}, regex=True, inplace = True)
df_test.replace({'family_status': 'вдовец / вдова'}, {'family_status': '2'}, inplace = True)
df_test.replace({'family_status': 'никогда не был(а) в браке'}, {'family_status': '3'}, inplace = True)
df_test.replace({'family_status': 'гражданский брак / проживание с партнером'}, {'family_status': '4'}, inplace = True)
df_test.replace({'family_status': 'раздельное проживание (официально не разведены)'}, {'family_status': '5'}, inplace = True)

# ethnos 1-европейская, 2-другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)
df_test.replace({'ethnos': 'европейская'}, {'ethnos': '1'}, regex=True, inplace = True)
df_test.replace({'ethnos': 'другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)'}, {'ethnos': '2'}, inplace = True)
df_test.replace({'ethnos': 'прочее (любая иная этно-расовая группа, не представленная выше)'}, {'ethnos': '2'},  inplace = True)

# nationality:
# 1-'Русские' 2-'Азербайджанцы' 3-'Татары' 4-'Немцы' 5-'Эстонцы'
# 0-'Другие национальности' 6-'Молдаване' 7-'Украинцы' 8-'Чуваши' 9-'Мордва'
# 10-'Киргизы' 11-'Казахи' 12-'Армяне' 13-'Белорусы' 14-'Таджики' 15-'Башкиры' 16-'Евреи'
# 17-'Буряты'
df_test.replace({'nationality': 'Другие национальности'}, {'nationality': '0'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Русские'}, {'nationality': '1'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Азербайджанцы'}, {'nationality': '2'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Татары'}, {'nationality': '3'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Немцы'}, {'nationality': '4'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Эстонцы'}, {'nationality': '5'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Молдаване'}, {'nationality': '6'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Украинцы'}, {'nationality': '7'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Чуваши'}, {'nationality': '8'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Мордва'}, {'nationality': '9'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Киргизы'}, {'nationality': '10'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Казахи'}, {'nationality': '11'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Армяне'}, {'nationality': '12'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Белорусы'}, {'nationality': '13'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Таджики'}, {'nationality': '14'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Башкиры'}, {'nationality': '15'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Евреи'}, {'nationality': '16'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Буряты'}, {'nationality': '17'}, regex=True, inplace = True)

# religion: 1-'Христианство' 2-'Атеист / агностик' 3-'Ислам' 0-'Нет'
df_test.replace({'religion': 'Нет'}, {'religion': '0'}, regex=True, inplace = True)
df_test.replace({'religion': 'Христианство'}, {'religion': '1'}, regex=True, inplace = True)
df_test.replace({'religion': 'Атеист / агностик'}, {'religion': '2'}, regex=True, inplace = True)
df_test.replace({'religion': 'Ислам'}, {'religion': '3'}, regex=True, inplace = True)

# education: 1-'3 - средняя школа / закон.среднее / выше среднего' 2-'5 - ВУЗ' 3-'2 - начальная школа' 4-'4 - профессиональное училище'
df_test.replace({'education': '3 - средняя школа / закон.среднее / выше среднего'}, {'education': '1'}, regex=True, inplace = True)
df_test.replace({'education': '5 - ВУЗ'}, {'education': '2'}, regex=True, inplace = True)
df_test.replace({'education': '2 - начальная школа'}, {'education': '3'}, regex=True, inplace = True)
df_test.replace({'education': '4 - профессиональное училище'}, {'education': '4'}, regex=True, inplace = True)

# profession: 0-'низкоквалифицированные работники' 1-'дипломированные специалисты'
# 2-'операторы и монтажники установок и машинного оборудования' 3-'служащие'
# 4-'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков'
# 5-'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры'
# 6-'техники и младшие специалисты' 7-'ведение домашнего хозяйства'
# 8-'квалифицированные работники сельского хозяйства и рыболовного'
# 9-'ремесленники и представители других отраслей промышленности'
# 10-'вооруженные силы'
df_test.replace({'profession': 'низкоквалифицированные работники'}, {'profession': '0'}, regex=True, inplace = True)
df_test.replace({'profession': 'дипломированные специалисты'}, {'profession': '1'}, regex=True, inplace = True)
df_test.replace({'profession': 'операторы и монтажники установок и машинного оборудования'}, {'profession': '2'}, regex=True, inplace = True)
df_test.replace({'profession': 'служащие'}, {'profession': '3'}, regex=True, inplace = True)
df_test.replace({'profession': 'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков'}, {'profession': '4'}, regex=True, inplace = True)
df_test.replace({'profession': 'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры'}, {'profession': '5'}, regex=True, inplace = True)
df_test.replace({'profession': 'техники и младшие специалисты'}, {'profession': '6'}, regex=True, inplace = True)
df_test.replace({'profession': 'ведение домашнего хозяйства'}, {'profession': '7'}, regex=True, inplace = True)
df_test.replace({'profession': 'квалифицированные работники сельского хозяйства и рыболовного'}, {'profession': '8'}, regex=True, inplace = True)
df_test.replace({'profession': 'ремесленники и представители других отраслей промышленности'}, {'profession': '9'}, regex=True, inplace = True)
df_test.replace({'profession': 'вооруженные силы'}, {'profession': '10'}, regex=True, inplace = True)

# smoke_status: 1-'Курит' 0-'Никогда не курил(а)' 2-'Бросил(а)'
df_test.replace({'smoke_status': 'Курит'}, {'smoke_status': '1'}, inplace = True)
df_test.replace({'smoke_status': 'Никогда не курил(а)'}, {'smoke_status': '0'},  inplace = True)
df_test.replace({'smoke_status': 'Бросил(а)'}, {'smoke_status': '2'},  inplace = True)


# pass_smoke_rate: 1-'1-2 раза в неделю' 2-'3-6 раз в неделю' 3-'не менее 1 раза в день' 4-'4 и более раз в день' 5-'2-3 раза в день'
df_test.replace({'pass_smoke_rate': '1-2 раза в неделю'}, {'pass_smoke_rate': '1'}, inplace = True)
df_test.replace({'pass_smoke_rate': '3-6 раз в неделю'}, {'pass_smoke_rate': '2'},  inplace = True)
df_test.replace({'pass_smoke_rate': 'не менее 1 раза в день'}, {'pass_smoke_rate': '3'},  inplace = True)
df_test.replace({'pass_smoke_rate': '4 и более раз в день'}, {'pass_smoke_rate': '4'},  inplace = True)
df_test.replace({'pass_smoke_rate': '2-3 раза в день'}, {'pass_smoke_rate': '5'},  inplace = True)

# alcohol_status, 1-'употребляю в настоящее время' 0-'никогда не употреблял' 2-'ранее употреблял'
df_test.replace({'alcohol_status': 'употребляю в настоящее время'}, {'alcohol_status': '1'}, inplace = True)
df_test.replace({'alcohol_status': 'никогда не употреблял'}, {'alcohol_status': '0'}, inplace = True)
df_test.replace({'alcohol_status': 'ранее употреблял'}, {'alcohol_status': '2'}, inplace = True)

In [ ]:
print(df_test)

### Удаление ненужных колонок

In [5]:
# drop id and id_y, sleep_start_time, sleep_end_time
columns_to_drop = ['id', 'id_y', 'sleep_start_time', 'sleep_end_time']
df_test.drop(labels=columns_to_drop, axis=1, inplace=True)

In [ ]:
for col in df_test:
    print(df_test[col], ' - ', df_test[col].unique())

### Нормализация датасета

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler() 
# scaled_values = scaler.fit_transform(df_test) 
# df_test.loc[:,:] = scaled_values

# say df is the dataframe
for c in df_test.columns:
    if df_test[c].dtype == np.float:
        df_test[c] = df_test[c].astype(int)

In [ ]:
df_test.head()
#df_test.describe()

In [ ]:
df_test.to_csv("processed.csv", sep=',', encoding='utf-8', index=False)

In [ ]:
df_test.dtypes

### Приведение типов к int

In [6]:
df_test['sex'] = df_test['sex'].astype(int)
df_test['family_status'] = df_test['family_status'].astype(int)
df_test['ethnos'] = df_test['ethnos'].astype(int)
df_test['nationality'] = df_test['nationality'].astype(int)
df_test['religion'] = df_test['religion'].astype(int)
df_test['education'] = df_test['education'].astype(int)
df_test['profession'] = df_test['profession'].astype(int)
df_test['smoke_status'] = df_test['smoke_status'].astype(int)
df_test['pass_smoke_rate'] = df_test['pass_smoke_rate'].astype(int)
df_test['alcohol_status'] = df_test['pass_smoke_rate'].astype(int)

### Определение средних значений по предсказываемым лейблам

In [11]:
print("arterial_hypertension: ", round(df_test.arterial_hypertension.value_counts()*100/len(df_test),2))
print("insult_status: ", round(df_test.insult_status.value_counts()*100/len(df_test),2))
print("infarction_status: ", round(df_test.infarction_status.value_counts()*100/len(df_test),2))
print("heart_failure: ", round(df_test.heart_failure.value_counts()*100/len(df_test),2))
print("oheart_diseases: ", round(df_test.oheart_diseases.value_counts()*100/len(df_test),2))

arterial_hypertension:  0    53.3
1    46.7
Name: arterial_hypertension, dtype: float64
insult_status:  0    95.71
1     4.29
Name: insult_status, dtype: float64
infarction_status:  0    87.75
1    12.25
Name: infarction_status, dtype: float64
heart_failure:  0    89.95
1    10.05
Name: heart_failure, dtype: float64
oheart_diseases:  0    90.99
1     9.01
Name: oheart_diseases, dtype: float64


### Формирование фреймов на основе предсказываемых лейблов

In [12]:
# 'Артериальная гипертензия': 'arterial_hypertension',
# 'ОНМК': 'insult_status',
# 'Стенокардия, ИБС, инфаркт миокарда': 'infarction_status',
# 'Сердечная недостаточность': 'heart_failure',
# 'Прочие заболевания сердца': 'oheart_diseases',

y1 = df_test["arterial_hypertension"]
y2 = df_test["insult_status"]
y3 = df_test["infarction_status"]
y4 = df_test["heart_failure"]
y5 = df_test["oheart_diseases"]
multiclass = ['arterial_hypertension','insult_status','infarction_status','heart_failure','oheart_diseases']
y = df_test[multiclass]
X1 = df_test.drop(multiclass, axis=1)
X2 = df_test.drop(multiclass, axis=1)
X3 = df_test.drop(multiclass, axis=1)
X4 = df_test.drop(multiclass, axis=1)
X5 = df_test.drop(multiclass, axis=1)


### SMOTE выравнивание несбалансированных классов

In [71]:
smote1 = SMOTE('auto',random_state=42)
X_train1_smote,y_train1_smote = smote1.fit_resample(X1,y1)

print("Before SMOTE1 :", Counter(y1))
print("After SMOTE1 :", Counter(y_train1_smote))

smote2 = SMOTE('auto',random_state=42)
X_train2_smote,y_train2_smote = smote2.fit_resample(X2,y2)

print("Before SMOTE2 :", Counter(y2))
print("After SMOTE2 :", Counter(y_train2_smote))

smote3 = SMOTE('auto',random_state=42)
X_train3_smote,y_train3_smote = smote3.fit_resample(X3,y3)

print("Before SMOTE3 :", Counter(y3))
print("After SMOTE3 :", Counter(y_train3_smote))

smote4 = SMOTE('auto',random_state=42)
X_train4_smote,y_train4_smote = smote4.fit_resample(X4,y4)

print("Before SMOTE4 :", Counter(y4))
print("After SMOTE4 :", Counter(y_train4_smote))

smote5 = SMOTE('auto',random_state=42)
X_train5_smote,y_train5_smote = smote5.fit_resample(X5,y5)

print("Before SMOTE5 :", Counter(y5))
print("After SMOTE5 :", Counter(y_train5_smote))

Before SMOTE1 : Counter({0: 509, 1: 446})
After SMOTE1 : Counter({0: 509, 1: 509})
Before SMOTE2 : Counter({0: 914, 1: 41})
After SMOTE2 : Counter({0: 914, 1: 914})
Before SMOTE3 : Counter({0: 838, 1: 117})
After SMOTE3 : Counter({0: 838, 1: 838})
Before SMOTE4 : Counter({0: 859, 1: 96})
After SMOTE4 : Counter({0: 859, 1: 859})
Before SMOTE5 : Counter({0: 869, 1: 86})
After SMOTE5 : Counter({0: 869, 1: 869})


In [60]:
# X_train1_smote,y_train1_smote = X1,y1
# X_train2_smote,y_train2_smote = X2,y2
# X_train3_smote,y_train3_smote = X3,y3
# X_train4_smote,y_train4_smote = X4,y4
# X_train5_smote,y_train5_smote = X5,y5

In [81]:
# 'arterial_hypertension','insult_status','infarction_status','heart_failure','oheart_diseases'

categorical_features_indices=[0]

######################################################################
# arterial_hypertension
print('Обучение arterial_hypertension....\n')

arterial_hypertension = CatBoostClassifier(verbose=False)
params = {
          'n_estimators': [100, 500, 1000],
          'depth': [3, 4, 5, 6, 7],
          'loss_function': ['MultiLogloss'],
          'l2_leaf_reg': np.logspace(-20, -19, 3, 7),
          'leaf_estimation_iterations': [10],
          'logging_level':['Silent'],
          'random_seed': [42]
         }
scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=arterial_hypertension, param_grid=params, scoring=scorer, cv=5)

# arterial_hypertension = CatBoostClassifier(learning_rate=0.055, 
#                           n_estimators=1000, 
#                           subsample=0.075, 
#                           max_depth=3, 
#                           verbose=False,
#                           l2_leaf_reg = 7,
#                           bootstrap_type="Bernoulli",
#                           loss_function='MultiLogloss')
#                           eval_metric='F1')

kf = StratifiedKFold(n_splits=4,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train1_smote,y_train1_smote)):
    tx = X_train1_smote.iloc[t_id]; ty = y_train1_smote.iloc[t_id]
    vx = X_train1_smote.iloc[v_id]; vy = y_train1_smote.iloc[v_id]
    arterial_hypertension.fit(tx,ty)        
    val_y = arterial_hypertension.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    recall_score = score(vy, val_y)
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score for arterial_hypertension {np.mean(f1)}")


Обучение arterial_hypertension....

fold 0 f1 0.673958338052202
[[81 36]
 [47 91]]
fold 1 f1 0.7450509869070919
[[93 31]
 [34 97]]
fold 2 f1 0.7204681081918934
[[92 36]
 [35 91]]
fold 3 f1 0.6919776119402985
[[81 32]
 [46 95]]
Mean f1 score for arterial_hypertension 0.7078637612728714


In [84]:
clf_grid.fit(X_train1_smote, y_train1_smote)
best_param = clf_grid.best_params_
best_param

CatBoostError: Only one of parameters ['verbose', 'logging_level', 'verbose_eval', 'silent'] should be set

### Обучение моделей для предсказания 'arterial_hypertension','insult_status','infarction_status','heart_failure','oheart_diseases'


In [87]:
# 'arterial_hypertension','insult_status','infarction_status','heart_failure','oheart_diseases'

categorical_features_indices=[0]

######################################################################
# arterial_hypertension
print('Обучение arterial_hypertension....\n')
arterial_hypertension = CatBoostClassifier(learning_rate=0.055, 
                          n_estimators=1000, 
                          subsample=0.075, 
                          max_depth=5, 
                          verbose=False,
                          l2_leaf_reg = 7,
                          bootstrap_type="Bernoulli",
                          loss_function='MultiLogloss')
                    #      eval_metric='F1')

kf = StratifiedKFold(n_splits=7,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train1_smote,y_train1_smote)):
    tx = X_train1_smote.iloc[t_id]; ty = y_train1_smote.iloc[t_id]
    vx = X_train1_smote.iloc[v_id]; vy = y_train1_smote.iloc[v_id]
    arterial_hypertension.fit(tx,ty)        
    val_y = arterial_hypertension.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    recall_score = score(vy, val_y)
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score for arterial_hypertension {np.mean(f1)}")

######################################################################
# insult_status
print('Обучение insult_status....\n')

insult_status = CatBoostClassifier(learning_rate=0.055, 
                          n_estimators=1000, 
                          subsample=0.075, 
                          max_depth=5, 
                          verbose=False,
                          l2_leaf_reg = 7,
                          bootstrap_type="Bernoulli",
                          #class_weights=[1, 1],
                          loss_function='MultiLogloss')
#                           eval_metric='F1')

kf = StratifiedKFold(n_splits=7,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train2_smote,y_train2_smote)):
    tx = X_train2_smote.iloc[t_id]; ty = y_train2_smote.iloc[t_id]
    vx = X_train2_smote.iloc[v_id]; vy = y_train2_smote.iloc[v_id]
    insult_status.fit(tx,ty)        
    val_y = insult_status.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score insult_status {np.mean(f1)}")

######################################################################
# infarction_status
print('Обучение infarction_status....\n')

infarction_status = CatBoostClassifier(learning_rate=0.055, 
                          n_estimators=1000, 
                          subsample=0.075, 
                          max_depth=5, 
                          verbose=False,
                          l2_leaf_reg = 7,
                          bootstrap_type="Bernoulli",
                          #class_weights=[1, 1],
                          loss_function='MultiLogloss')
#                           eval_metric='F1')

kf = StratifiedKFold(n_splits=7,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train3_smote,y_train3_smote)):
    tx = X_train3_smote.iloc[t_id]; ty = y_train3_smote.iloc[t_id]
    vx = X_train3_smote.iloc[v_id]; vy = y_train3_smote.iloc[v_id]
    infarction_status.fit(tx,ty)        
    val_y = infarction_status.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score infarction_status {np.mean(f1)}")

######################################################################
# heart_failure
print('Обучение heart_failure....\n')

heart_failure = CatBoostClassifier(learning_rate=0.055, 
                          n_estimators=1000, 
                          subsample=0.075, 
                          max_depth=5, 
                          verbose=False,
                          l2_leaf_reg = 7,
                          bootstrap_type="Bernoulli",
                          #class_weights=[1, 1],
                          loss_function='MultiLogloss')
#                           eval_metric='F1')

kf = StratifiedKFold(n_splits=7,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train4_smote,y_train4_smote)):
    tx = X_train4_smote.iloc[t_id]; ty = y_train4_smote.iloc[t_id]
    vx = X_train4_smote.iloc[v_id]; vy = y_train4_smote.iloc[v_id]
    heart_failure.fit(tx,ty)        
    val_y = heart_failure.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score heart_failure {np.mean(f1)}")

######################################################################
# oheart_diseases
print('Обучение oheart_diseases....\n')

oheart_diseases = CatBoostClassifier(learning_rate=0.055, 
                          n_estimators=1000, 
                          subsample=0.075, 
                          max_depth=5, 
                          verbose=False,
                          l2_leaf_reg = 7,
                          bootstrap_type="Bernoulli",
                          #class_weights=[1, 1],
                          loss_function='MultiLogloss')
#                           eval_metric='F1')

kf = StratifiedKFold(n_splits=7,shuffle=True,random_state=99)
f1 = []

for fold,(t_id,v_id) in enumerate(kf.split(X_train5_smote,y_train5_smote)):
    tx = X_train5_smote.iloc[t_id]; ty = y_train5_smote.iloc[t_id]
    vx = X_train5_smote.iloc[v_id]; vy = y_train5_smote.iloc[v_id]
    oheart_diseases.fit(tx,ty)        
    val_y = oheart_diseases.predict(vx)
    
    F1_score = f1_score(vy, val_y,average='weighted')
    f1.append(F1_score)

    print(f"fold {fold} f1 {F1_score}")
    print(confusion_matrix(val_y, vy))

print(f"Mean f1 score oheart_diseases {np.mean(f1)}")

Обучение arterial_hypertension....

fold 0 f1 0.6432330827067669
[[44 23]
 [29 50]]
fold 1 f1 0.6777041942604857
[[47 21]
 [26 52]]
fold 2 f1 0.7259759759759759
[[52 19]
 [21 54]]
fold 3 f1 0.6934880636604774
[[58 29]
 [15 43]]
fold 4 f1 0.7234014571661537
[[49 16]
 [24 56]]
fold 5 f1 0.7170798721730095
[[50 19]
 [22 54]]
fold 6 f1 0.6551724137931034
[[47 25]
 [25 48]]
Mean f1 score for arterial_hypertension 0.6908650085337102
Обучение insult_status....

fold 0 f1 0.9618298368298368
[[125   4]
 [  6 127]]
fold 1 f1 0.9655071150277589
[[124   2]
 [  7 128]]
fold 2 f1 0.9655071150277589
[[124   2]
 [  7 128]]
fold 3 f1 0.9731722005771457
[[125   1]
 [  6 129]]
fold 4 f1 0.9693459589209388
[[125   3]
 [  5 128]]
fold 5 f1 0.9655162289225486
[[125   4]
 [  5 127]]
fold 6 f1 0.957853168683115
[[124   5]
 [  6 126]]
Mean f1 score insult_status 0.9655330891413003
Обучение infarction_status....

fold 0 f1 0.9166608792277241
[[109   9]
 [ 11 111]]
fold 1 f1 0.8874042153915658
[[103  10]
 [ 17 1

In [ ]:
# accuracy_per_class = eval_metric(Y_test, test_predict, 'Accuracy:type=PerClass')
# for cls, value in zip(clf.classes_, accuracy_per_class):
#     print(f'Accuracy for class {cls}: {value}')

### Приведение тестового датасета в анализируемый вид

In [16]:
df_test = pd.read_csv('D:/Projects/cp_heartdetect_model/Dataset/test.csv')

df_test = df_test.rename(columns=
{'ID': 'id', 
'Пол': 'sex',
'Семья': 'family_status',
'Этнос': 'ethnos',
'Национальность': 'nationality',
'Религия': 'religion',
'Образование': 'education',
'Профессия': 'profession',
'Вы работаете?': 'work_status',
'Выход на пенсию': 'pension_age',
'Прекращение работы по болезни': 'stop_working',
'Сахарный диабет': 'diabetes',
'Гепатит': 'hepatitis',
'Онкология': 'oncology',
'Хроническое заболевание легких': 'lung_disease',
'Бронжиальная астма': 'bronchial_asthma',
'Туберкулез легких ': 'tuberculosis',
'ВИЧ/СПИД': 'HIV_AIDS',
'Регулярный прим лекарственных средств': 'intake_medicines',
'Травмы за год': 'injuries_per_year',
'Переломы': 'fractures',
'Статус Курения': 'smoke_status',
'Возраст курения': 'smoking_age',
'Сигарет в день': 'cigarettes_pd',
'Пассивное курение': 'passive_smoke',
'Частота пасс кур': 'pass_smoke_rate',
'Алкоголь': 'alcohol_status',
'Возраст алког': 'alcohol_age',
'Время засыпания': 'sleep_start_time',
'Время пробуждения': 'sleep_end_time',
'Сон после обеда': 'sleep_daily',
'Спорт, клубы': 'sport_status',
'Религия,клубы': 'religion_status',
'Артериальная гипертензия': 'arterial_hypertension',
'ОНМК': 'insult_status',
'Стенокардия, ИБС, инфаркт миокарда': 'infarction_status',
'Сердечная недостаточность': 'heart_failure',
'Прочие заболевания сердца': 'oheart_diseases',
})

# replace empty with 0
#df_test = df_test.replace('', np.nan, regex=True)
df_test = df_test.fillna(0)

# sex 1-женщина, 0-мужчина
df_test.replace({'sex': 'М'}, {'sex': '0'}, regex=True, inplace = True)
df_test.replace({'sex': 'Ж'}, {'sex': '1'}, regex=True, inplace = True)

# family_status: 1-в браке в настоящее время, 0-в разводе, 2-вдовец / вдова, 3-никогда не был(а) в браке, 4-гражданский брак / проживание с партнером, 5-раздельное проживание (официально не разведены)
df_test.replace({'family_status': 'в браке в настоящее время'}, {'family_status': '1'}, regex=True, inplace = True)
df_test.replace({'family_status': 'в разводе'}, {'family_status': '0'}, regex=True, inplace = True)
df_test.replace({'family_status': 'вдовец / вдова'}, {'family_status': '2'}, inplace = True)
df_test.replace({'family_status': 'никогда не был(а) в браке'}, {'family_status': '3'}, inplace = True)
df_test.replace({'family_status': 'гражданский брак / проживание с партнером'}, {'family_status': '4'}, inplace = True)
df_test.replace({'family_status': 'раздельное проживание (официально не разведены)'}, {'family_status': '5'}, inplace = True)

# ethnos 1-европейская, 2-другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)
df_test.replace({'ethnos': 'европейская'}, {'ethnos': '1'}, regex=True, inplace = True)
df_test.replace({'ethnos': 'другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)'}, {'ethnos': '2'}, inplace = True)
df_test.replace({'ethnos': 'прочее (любая иная этно-расовая группа, не представленная выше)'}, {'ethnos': '2'},  inplace = True)

# nationality:
# 1-'Русские' 2-'Азербайджанцы' 3-'Татары' 4-'Немцы' 5-'Эстонцы'
# 0-'Другие национальности' 6-'Молдаване' 7-'Украинцы' 8-'Чуваши' 9-'Мордва'
# 10-'Киргизы' 11-'Казахи' 12-'Армяне' 13-'Белорусы' 14-'Таджики' 15-'Башкиры' 16-'Евреи'
# 17-'Буряты'
df_test.replace({'nationality': 'Другие национальности'}, {'nationality': '0'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Русские'}, {'nationality': '1'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Азербайджанцы'}, {'nationality': '2'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Татары'}, {'nationality': '3'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Немцы'}, {'nationality': '4'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Эстонцы'}, {'nationality': '5'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Молдаване'}, {'nationality': '6'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Украинцы'}, {'nationality': '7'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Чуваши'}, {'nationality': '8'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Мордва'}, {'nationality': '9'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Киргизы'}, {'nationality': '10'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Казахи'}, {'nationality': '11'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Армяне'}, {'nationality': '12'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Белорусы'}, {'nationality': '13'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Таджики'}, {'nationality': '14'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Башкиры'}, {'nationality': '15'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Евреи'}, {'nationality': '16'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Буряты'}, {'nationality': '17'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Удмурты'}, {'nationality': '18'}, regex=True, inplace = True)
df_test.replace({'nationality': 'Лезгины'}, {'nationality': '19'}, regex=True, inplace = True)

# religion: 1-'Христианство' 2-'Атеист / агностик' 3-'Ислам' 0-'Нет'
df_test.replace({'religion': 'Нет'}, {'religion': '0'}, regex=True, inplace = True)
df_test.replace({'religion': 'Христианство'}, {'religion': '1'}, regex=True, inplace = True)
df_test.replace({'religion': 'Атеист / агностик'}, {'religion': '2'}, regex=True, inplace = True)
df_test.replace({'religion': 'Ислам'}, {'religion': '3'}, regex=True, inplace = True)
df_test.replace({'religion': 'Другое'}, {'religion': '0'}, regex=True, inplace = True)
df_test.replace({'religion': 'Индуизм'}, {'religion': '4'}, regex=True, inplace = True)

# education: 1-'3 - средняя школа / закон.среднее / выше среднего' 2-'5 - ВУЗ' 3-'2 - начальная школа' 4-'4 - профессиональное училище'
df_test.replace({'education': '3 - средняя школа / закон.среднее / выше среднего'}, {'education': '1'}, regex=True, inplace = True)
df_test.replace({'education': '5 - ВУЗ'}, {'education': '2'}, regex=True, inplace = True)
df_test.replace({'education': '2 - начальная школа'}, {'education': '3'}, regex=True, inplace = True)
df_test.replace({'education': '4 - профессиональное училище'}, {'education': '4'}, regex=True, inplace = True)

# profession: 0-'низкоквалифицированные работники' 1-'дипломированные специалисты'
# 2-'операторы и монтажники установок и машинного оборудования' 3-'служащие'
# 4-'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков'
# 5-'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры'
# 6-'техники и младшие специалисты' 7-'ведение домашнего хозяйства'
# 8-'квалифицированные работники сельского хозяйства и рыболовного'
# 9-'ремесленники и представители других отраслей промышленности'
# 10-'вооруженные силы'
df_test.replace({'profession': 'низкоквалифицированные работники'}, {'profession': '0'}, regex=True, inplace = True)
df_test.replace({'profession': 'дипломированные специалисты'}, {'profession': '1'}, regex=True, inplace = True)
df_test.replace({'profession': 'операторы и монтажники установок и машинного оборудования'}, {'profession': '2'}, regex=True, inplace = True)
df_test.replace({'profession': 'служащие'}, {'profession': '3'}, regex=True, inplace = True)
df_test.replace({'profession': 'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков'}, {'profession': '4'}, regex=True, inplace = True)
df_test.replace({'profession': 'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры'}, {'profession': '5'}, regex=True, inplace = True)
df_test.replace({'profession': 'техники и младшие специалисты'}, {'profession': '6'}, regex=True, inplace = True)
df_test.replace({'profession': 'ведение домашнего хозяйства'}, {'profession': '7'}, regex=True, inplace = True)
df_test.replace({'profession': 'квалифицированные работники сельского хозяйства и рыболовного'}, {'profession': '8'}, regex=True, inplace = True)
df_test.replace({'profession': 'ремесленники и представители других отраслей промышленности'}, {'profession': '9'}, regex=True, inplace = True)
df_test.replace({'profession': 'вооруженные силы'}, {'profession': '10'}, regex=True, inplace = True)

# smoke_status: 1-'Курит' 0-'Никогда не курил(а)' 2-'Бросил(а)'
df_test.replace({'smoke_status': 'Курит'}, {'smoke_status': '1'}, inplace = True)
df_test.replace({'smoke_status': 'Никогда не курил(а)'}, {'smoke_status': '0'},  inplace = True)
df_test.replace({'smoke_status': 'Никогда не курил'}, {'smoke_status': '0'},  inplace = True)
df_test.replace({'smoke_status': 'Бросил(а)'}, {'smoke_status': '2'},  inplace = True)


# pass_smoke_rate: 1-'1-2 раза в неделю' 2-'3-6 раз в неделю' 3-'не менее 1 раза в день' 4-'4 и более раз в день' 5-'2-3 раза в день'
df_test.replace({'pass_smoke_rate': '1-2 раза в неделю'}, {'pass_smoke_rate': '1'}, inplace = True)
df_test.replace({'pass_smoke_rate': '3-6 раз в неделю'}, {'pass_smoke_rate': '2'},  inplace = True)
df_test.replace({'pass_smoke_rate': 'не менее 1 раза в день'}, {'pass_smoke_rate': '3'},  inplace = True)
df_test.replace({'pass_smoke_rate': '4 и более раз в день'}, {'pass_smoke_rate': '4'},  inplace = True)
df_test.replace({'pass_smoke_rate': '2-3 раза в день'}, {'pass_smoke_rate': '5'},  inplace = True)

# alcohol_status, 1-'употребляю в настоящее время' 0-'никогда не употреблял' 2-'ранее употреблял'
df_test.replace({'alcohol_status': 'употребляю в настоящее время'}, {'alcohol_status': '1'}, inplace = True)
df_test.replace({'alcohol_status': 'никогда не употреблял'}, {'alcohol_status': '0'}, inplace = True)
df_test.replace({'alcohol_status': 'ранее употреблял'}, {'alcohol_status': '2'}, inplace = True)

# drop id and id_y, sleep_start_time, sleep_end_time
columns_to_drop = ['id', 'sleep_start_time', 'sleep_end_time']
df_test.drop(labels=columns_to_drop, axis=1, inplace=True)

df_test.head()

,sex,family_status,ethnos,nationality,religion,education,profession,work_status,pension_age,stop_working,...,smoke_status,smoking_age,cigarettes_pd,passive_smoke,pass_smoke_rate,alcohol_status,alcohol_age,sleep_daily,sport_status,"Религия, клубы"
0,1,2,1,1,1,1,9,1,0,0,...,0,0.0,0.0,0,0,1,23.0,0,1,1
1,1,0,1,1,1,2,1,0,1,0,...,0,0.0,0.0,0,0,1,22.0,0,0,0
2,1,1,1,1,1,2,1,0,1,0,...,0,0.0,0.0,0,0,1,18.0,0,0,0
3,1,1,1,1,1,4,0,1,0,0,...,0,0.0,0.0,0,0,0,0.0,0,0,0
4,0,1,1,1,1,1,2,0,1,0,...,1,14.0,20.0,0,0,1,19.0,0,0,0


In [62]:
'arterial_hypertension','insult_status','infarction_status','heart_failure','oheart_diseases'

arterial_hypertension_model = arterial_hypertension.predict(df_test)
arterial_hypertension_model

insult_status_model = insult_status.predict(df_test)
insult_status_model

infarction_status_model = infarction_status.predict(df_test)
infarction_status_model

heart_failure_model = heart_failure.predict(df_test)
heart_failure_model

oheart_diseases_model = oheart_diseases.predict(df_test)
oheart_diseases_model

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [69]:
import sklearn
sklearn.metrics.recall_score(y1,arterial_hypertension.predict(df_test),average=None)

ValueError: Found input variables with inconsistent numbers of samples: [955, 638]

### Генерация решения

In [63]:
# Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца

df_val = pd.read_csv('D:/Projects/cp_heartdetect_model/Dataset/test.csv')

submission = pd.DataFrame(df_val['ID'],columns=['ID',])
submission['Артериальная гипертензия'] = arterial_hypertension_model
submission['ОНМК'] = insult_status_model
submission["\"Стенокардия, ИБС, инфаркт миокарда\""] = infarction_status_model
submission['Сердечная недостаточность'] = heart_failure_model
submission['Прочие заболевания сердца'] = oheart_diseases_model

submission

,ID,Артериальная гипертензия,ОНМК,"""Стенокардия, ИБС, инфаркт миокарда""",Сердечная недостаточность,Прочие заболевания сердца
0,54-001-019-01,1,0,0,0,0
1,54-002-133-01,1,0,0,0,0
2,54-001-007-01,0,0,0,0,0
3,54-102-116-01,0,0,0,0,0
4,54-502-005-02,1,0,0,0,0
...,...,...,...,...,...,...
633,54-102-095-01,1,0,0,0,0
634,54-102-235-01,1,0,0,0,0
635,54-502-016-01,1,0,0,0,0
636,54-002-138-01,0,0,0,0,0


In [64]:
print('Артериальная гипертензия', submission['Артериальная гипертензия'].value_counts())
print('ОНМК', submission['ОНМК'].value_counts())
print('"Стенокардия, ИБС, инфаркт миокарда"', submission['"Стенокардия, ИБС, инфаркт миокарда"'].value_counts())
print('Сердечная недостаточность', submission['Сердечная недостаточность'].value_counts())
print('Прочие заболевания сердца', submission['Прочие заболевания сердца'].value_counts())

Артериальная гипертензия 0    353
1    285
Name: Артериальная гипертензия, dtype: int64
ОНМК 0    634
1      4
Name: ОНМК, dtype: int64
"Стенокардия, ИБС, инфаркт миокарда" 0    628
1     10
Name: "Стенокардия, ИБС, инфаркт миокарда", dtype: int64
Сердечная недостаточность 0    633
1      5
Name: Сердечная недостаточность, dtype: int64
Прочие заболевания сердца 0    634
1      4
Name: Прочие заболевания сердца, dtype: int64


In [65]:
submission.to_csv('submission.csv', encoding='utf-8', index=False)

In [66]:
with open('submission.csv', 'r', encoding='utf-8') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"""', '"')

with open('submission.csv', 'w', encoding='utf-8') as file:
  file.write(filedata)